$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

**Answer:** "receptive field" is the name for the size of the input that the kernel "see" and can produce a feature out of.
The most important meaning is when we use a CNN, the conv component extract features, and we want to be able to extract the most important features for the given problem (in general images we want to be scale invariant and so we want the end of the CNN to have a large receptive field).


2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:**
There are several ways to increase the receptive field, we will talk about **kernel size**, **stride**, and **dilated convulution**.
(note that the easiest way to control the R.F is to add layers, but the question ask about the growth from layer to layer)

$RF_L =  RF_{L-1} + (F_L -1) \times S $ where L denote a layer,F the kernel size and S denote stride.
(the real formula of r.f is $RF_L = RF_{L-1} + (F_L -1)\cdot \prod_{l=1}^{i-1} S_l$ but the effect of stride is enogh on one layer for this answer).

1. kernel size: we can see from the eq. above that bigger kernel size provide bigger receptive field. this is the most intuitive, since we simply see more pixels.
2. stride: assuming we pad same, stride denotes the 'step' size we do while convolve.
    *for example for 1 layer with ker_size 3x3 and stride 2: the first multiplication would see prev_layer[0:2][0:2], then prev_layer[2:4][0:2], then prev_layer[4:6][0:2], then prev_layer[0:2][2:4] and so on... in total the receptive field would be 7x7*
3. dilated convulution: this notion simply make a 'gap' (or add zeros) between the kernel parameters. for kernel_size 3x3 and dilation=2, we would get a kernel of 9 parameters, but the receptive field would be the same like we used kernel of 5x5.


3. Imagine a CNN with three convolutional layers, defined as follows:

In [2]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

C:\Users\Admin\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:** as we shown in the equation above and for  dialution $rf_{d\_i} = d\cdot (f_i-1) + 1 $

$rf_1 = 3$  
$rf_2 = 3 + 1\cdot1 = 4$  
$rf_3 = 4 + 4\cdot2 = 12$  
$rf_4 = 12 + 1\cdot(2\cdot2) = 16$  
$rf_5 = 16 + 12\cdot8 = 112$

so final receptive field is `112x112`

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

**Answer:** The reason is, that a residual layer and conv layer need to learn diffrent functions, since the input is diffrent.
The network could have the exect same preformance with diffrent filters because of that reason. let's use an example:
We have a network that learned all the featurse needed by layer i, and the network dept is i+1.
the i+1 layer would need to learn the idendity funtion. while for regular convolution the filter would be close to $f_{i+1}(\vec{x};\vec{\theta}_l) = \vec{I}$ and $\vec{y}_i = \vec{x} f_{i+1} =\vec{x} \vec{I} = \vec{x}$, the residual have the input x, so it would need to learn $f_{i+1}(\vec{x};\vec{\theta}_l) = \vec{0}$ and $\vec{y}_i+1 = \vec{x} f_{i+1} + \vec{x} =\vec{x} \vec{0} +\vec{x} = \vec{x}$.

### Dropout

1. Consider the following neural network:

In [ ]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

**answer**: the probability of neuron to drop afer one layer of dropout is `p1` ,so we have `(1-p1)` left and then we apply another dropout with `p2`, then we stay with  `(1-p1)(1-p2)` that is `1+p1p2 -p1 -p2` and q is the probability to drop (we calculated 1-q), means `q = p1+p2 - p1p2`

2. **True or false**: dropout must be placed only after the activation function.
**answer**: False: there is no logical effect on using dropout before activisions, but activision function work on all the input, so the library implementation would save some computation time if we use if after the activision

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

**answer**: recalls for expectation $E(x) = p(x)  x$

let's look at the neurons as a vector $\vec{X}$ with probability 1 to pass forward and $E=X$.
When we use dropout of $p$,the probability to choose x is (1-p) and $E(X) = (1-p)X$.
the activision expect the same expectation, so we use $1/(1-p) E(X) = (1-p)X/(1-p) = X$

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

**answer**: L2 is not commonly use for classifications, while we can teoretically use it for binary classification, it won't be wise.
$L2 = \frac{1}{n}\sum_n (y_i - \hat y_i^p)^2$.
the main problem with using it is, we take into considiration only the final prediction but not the "score" (probability- the softmax output) of the prediction and there will be no notion of ceirtenty in the classification.
while L2 valuse would be 1 or 0, other score, like BCE (Binary Cross Entropy) would consider probability of classification into consideration: $CE =- \frac{1}{n}\sum_n  y log(p(\hat{y})) + (1-y)log(1-p(\hat{y})))$

example: for one image of a dog and classification score of p(dog)=0.51 and p(hotdog)=0.49,
$L2 = 0$  and $CE = -log(0.51) = 0.29$.
we know of course, that a model that think of a dog and hotdog so close, need to be optimized some more.
the L2 optimizer model look at the example the same way as in other example of p(dog)=1 and p(hotdog)=0. 



2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [1]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

**Answer**: it's look like the prediction of the global average temperture as a function of the number of pirates is a simple function.
the suggested mlpirate model is very large and severely suffer from severely vanishing gradients and if not, very likely overfitting. 

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.
**Answer**: `tanh` behave like a linear function for low values of $x$, the main reason to use it is to avoid exploding gradients problem, so we don't belive this is the right fit for the given problem

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
    1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
    2. The gradient of ReLU is linear with its input when the input is positive.
    3. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

**Answer**:

    A. false: ReLU small values in input behave as small values in output, it's in common use for the fast gradients calculation and simplicity.
    B. false: the function is liniar with positive input, the gradient is constant. 
    C. ture in a way*: it is true that for a given layer with negative output the neuron would not recive gradient update, since the gradient for negative input of ReLu is 0, but not teoretically, we train on a lot of data and to be a "dead" neuron, the same one should get negative values for all samples (even in the batched G.D, we need all the data to produce negative output for the input of the ReLU). this can be, but negative values do not get to be more negative since there are no gradients, so in summery, teoretically true.

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer:**<br/>
Regular Gradient Descent is an optimization algorithm calculating the gradient of the loss over a given dataset,
while Stochastic Gradient Descent samples one example at each iteration and calculates the gradient of the loss for that one example.
Mini-batch SGD can be viewed as a mix between SGD and regular GD as it it a method in which a fixed batch of examples is independently sampled from the dataset,
and the gradient of the loss is calculated over this fixed-size batch.

2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

**Answer:**<br/>
A. The first reason SGD is more often used in practice than GD is that many datasets are extremely large, and thus it is impractical to calculate the gradient over all of the dataset as it will both take a lot of time and take a lot of memory.
The second reason is that many times sampling different examples from the dataset independently might help the model to generelize better, it will also generate a noisier gradient and that will help the optimization process escape local minima.

B. Regular Gradient Descent can not be used at all when the dataset is so large that the storage requirements of the algorithm are not avaliable.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**TODO:COULD EXPLAIN MORE IN THIS ANSWER!!!!**

**Answer:**<br/>
We would expect the number of iterations required to converge to ${l_0}$ to decrease as using batch size B, we would train over B examples in each iteration, in n iterations we trained our model over ${n\times B}$ samples, increasing the batch size to 2B would allow us to train over twice as many examples and thus, would allow our model function to be approximately more similiar to the optimal function of the classifier, thus we would expect our optimization to converge faster to loss ${l_0}$.

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

**Answer:**<br/>
A. False. Using SGD, each epoch we would sample an example from the dataset and perform an optimization step on that sample, not on the entire dataset.<br/>
B. False. Gradients with SGD are varying not only since of the step that the model has performed, but since of the difference of the samples the gradients are calculated on, thus different SGD gradients usually would have larger variance than regular GD gradients, since that they would be harder to converge to minima, and will usually take more iterations to approximate the target function than GD.<br/>
C. True. SGD gradients are more varying and thus if the function reaches a local minima i.e zero gradient, a different sample could have a non-zero gradient that would let the optimization escape of that local minima.
GD in contrast has no such option as the gradient is always calculated over all the dataset.<br/>
D. False. SGD trains with one sample at each epoch and thus has to keep memory of only the calculations for this sample, GD in contrast would calculate the gradient for all of the examples in the dataset in parallel and thus would have to use more memory for these calculations.<br/>
E. False. A learning algorithm converges when it reaches a small gradient, usually this would be a local minimum, but this could also be a saddle point, as we explained SGD is more likely to escape non-global minimums, but it is not guaranteed to escape, GD is a fortiori not guaranteed to converge to global minimum for non-convex optimization as it has less chance to escape local minimum.<br/>
F. False. Newton's method uses the curvature information to calculate a direction which will satisfy Hd=-g (where H is the hessian i.e matrix of second order derivatives which represents curvature, d is the descent direction and g is the gradient i.e the first order derivative of the function) thus, for functions with high curvature newton's method will take the curvature into considiration and take a more direct route while SGD with momentum will take the route of the gradient which is tangent to the function plus the momentum which is dependant on the previous gradients, thus the route of Newton's method could converge faster.<br/>
Example  (Quadratic function): for ${f(x) = x^tAx, \lambda_{max}, \lambda_{min}}$ are the greater and smaller eigenvalues of A.<br/>We define ${\delta_k=f(x_k)-f(x^*)}$ where ${x_k}$ is the value of x at iteration k, and ${x^*}$ is the optimal x, ${c=1-\lambda_{min}/\lambda_{max}}$ is defined as the convergence rate of the problem and so the convergence of Newton's method will be quadratic so ${\delta_{k+1} \leq c\delta_k^2}$ while for GD and SGD only Linear convergence is guaranteed:{

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

**Answer:**<br/>
**False**: In order for the outer problem to be solved (i.e training the network), the inner problem must be solvable either through a descent based algorithm or through other methods (could be exact analytical solution). <br/>
Example: for the network we used in the tutorial, we will select ${f(y;z)=||Ay-z||_2^2}$. Thus, ${\hat{y}=argmin_yf(y;z)=(A^tA)^{-1}A^tz}$ is the solution of Least Squares problem. Thus the inner problem can be solved analytically and not throught descent based algorithm<br/><center><img src="https://nbviewer.jupyter.org/github/vistalab-technion/cs236781-tutorials/blob/master/t05/imgs/bilevel.png"/></center> 

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

**Answer:**<br/>
A. Vanishing gradient is a situation in which during a gradient based optimization of a neural network, the calculated gradient of the network is very small. Since gradient based optimization updates the model proportionally to the gradient of the model, the vanishing gradients create a problem which halts the optimization of the model through very small gradient.<br/>Exploding gradient is a situation in which the the gradient of the network is extermely large, thus the optimization of the model through gradient descent based algorithms could have a difficulty converging because of the large gradient.<br/>
B. The gradient of a neural network is calculated through the chain rule, and thus a deep network could lead to vanishing gradient if the partial derivative of a large number of layers has a spactral norm smaller than 1, if all of the layers satisfy this condition we are guaranteed to have vanishing gradients for a feedforward network and a large enough number of layers, if the spactral norm of the partial derivative of all the layers is greater than 1 we may encounter exploding gradients.<br/>
C. We will take an example of an RNN, and calculate gradient of the loss at time t over the parameters of the model at time i (t > i),
<h3>${\frac{\delta l_t}{\delta\Theta_i} = \frac{\delta l_t}{\delta y_t}\frac{\delta y_t }{\delta h_t}\frac{\delta h_t }{\delta h_i }\frac{\delta^+ h_i}{\delta \Theta_i} = \frac{\delta l_t}{\delta y_t}\frac{\delta y_t }{\delta h_t}\frac{\delta h_t }{\delta h_{t-1} }...\frac{\delta h_{i+1}}{\delta h_i}\frac{\delta^+ h_i}{\delta \Theta_i} ,\text{  } \frac{\delta y_t }{\delta h_t} = W_{hy}^t, \text{ }\frac{\delta h_t }{\delta h_{t-1}}=W_{hh}^t \Phi'(z_t)}$</h3> Thus, <h3>${\frac{\delta y_t }{\delta h_t}\frac{\delta h_t }{\delta h_i }\frac{\delta^+ h_i}{\delta \Theta_i} = W_{hy}^tW_{hh}^t\Phi'_t...W_{hh}^t\Phi'_{i+1}}$</h3>
If we assume ${\Phi'=I ,W_{hh} \leq I \cdot 0.9}$, we will get ${lim_{t\rightarrow \infty}\frac{\delta l_t}{\delta\Theta_i} = 0}$.<br/>
For ${\Phi'=I ,W_{hh} \geq I \cdot 1.1}$ we will get  ${lim_{t\rightarrow \infty}\frac{\delta l_t}{\delta\Theta_i} = \infty}$.<br/>
<br/>
D. If we would be able to look at the parameters tensor, we could estimate the gradient as the difference of the parameters at different iterations.<br/>
Otherwise, if we look at the discrapancy (the loss) of the model during optimization, we could estimate that if the discrapancy is statis i.e does not change over iterations at all, we could assume we have a problem of vanishign gradients, if the discrapancy is bouncy (i.e the loss changes a lot but does not converge), we could assume we have a problem of exploding gradients as the parameters are changing much.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

**Answer:**<br/>
We will mark: ${z_1 = W_1x+b_1, z2 = \phi(z1), \hat{y}=W_2z_2+b_2}$<br/>
${L = \sum_{i=1}^{N}l(y^i,\hat{y^i}), L_R = \lambda/2(||W_1||_F^2 + ||W_2||_F^2)}$ ,Now we can derive the partial derivatives:<br/>
${\frac{\delta z_1}{\delta x} = W_1 , \frac{\delta z_1}{\delta W_1} = x, \frac{\delta z_1}{\delta b_1} = 1}$ <br/>
${\frac{\delta z_2}{\delta z_1} = diag(\frac{\delta \phi(z_1)}{\delta z_1})=\Phi'(z_1)}$<br/>
${\frac{\delta \hat{y}}{\delta z_2} = W_2, \frac{\delta \hat{y}}{\delta W_2} = z_2, \frac{\delta \hat{y}}{\delta b_2} = 1}$<br/>
${\frac{\delta l}{\delta \hat{y}} = -y/\hat{y} + (1-y)/(1-\hat{y})}$<br/>
${\frac{\delta L}{\delta l} = 1/N, \frac{\delta L_R}{\delta W_1} = 2W_1 \lambda/2 = \lambda W_1, \frac{\delta L_R}{\delta W_2} = 2W_2 \lambda/2 = \lambda W_2}$<br/>
By the chain rule:<br/>
${\frac{\delta L_S}{\delta W_1} = \sum_{i=1}^{N}(\frac{\delta L}{\delta l(y^i,\hat{y}^i)}\frac{\delta l(y^i,\hat{y}^i)}{\delta \hat{y}}\frac{\delta \hat{y}}{\delta z_2}\frac{\delta z_2}{\delta z_1}\frac{\delta z_1}{\delta W_1}) + \frac{\delta L_R}{\delta W_1} = 1/N \sum_{i=1}^{N}(((-y/\hat{y} + (1-y)/(1-\hat{y})) \cdot W_2 \cdot \Phi'(z_1) \cdot x) + \lambda W_1}$<br/>
${\frac{\delta L_S}{\delta b_1} = \frac{\delta L}{\delta b_1} = \sum_{i=1}^{N}\frac{\delta L}{\delta l(y^i,\hat{y}^i)}\frac{\delta l(y^i,\hat{y}^i)}{\delta \hat{y}}\frac{\delta \hat{y}}{\delta z_2}\frac{\delta z_2}{\delta z_1}\frac{\delta z_1}{\delta b_1} = 1/N\sum_{i=1}^{N}(((-y/\hat{y} + (1-y)/(1-\hat{y})) \cdot W_2 \cdot \Phi'(z_1))}$<br/>
${\frac{\delta L_S}{\delta W_2} = \sum_{i=1}^{N}(\frac{\delta L}{\delta l(y^i,\hat{y}^i)}\frac{\delta l(y^i,\hat{y}^i)}{\delta \hat{y}}\frac{\delta \hat{y}}{\delta W_2}) + \frac{\delta L_R}{\delta W_2} = 1/N \sum_{i=1}^{N}(((-y/\hat{y} + (1-y)/(1-\hat{y}))\cdot \phi(z_1)) + \lambda W_2}$<br/>
${\frac{\delta L_S}{\delta b_2} = \frac{\delta L}{\delta b_2} = \sum_{i=1}^{N}\frac{\delta L}{\delta l(y^i,\hat{y}^i)}\frac{\delta l(y^i,\hat{y}^i)}{\delta \hat{y}}\frac{\delta \hat{y}}{\delta b_2} = 1/N\sum_{i=1}^{N}((-y/\hat{y} + (1-y)/(1-\hat{y}))}$<br/>
${\frac{\delta L_S}{\delta x} = \sum_{i=1}^{N}(\frac{\delta L}{\delta l(y^i,\hat{y}^i)}\frac{\delta l(y^i,\hat{y}^i)}{\delta \hat{y}}\frac{\delta \hat{y}}{\delta z_2}\frac{\delta z_2}{\delta z_1}\frac{\delta z_1}{\delta x}) = 1/N \sum_{i=1}^{N}(((-y/\hat{y} + (1-y)/(1-\hat{y})) \cdot W_2 \cdot \Phi'(z_1) \cdot W_1)}$<br/>

2. The derivative of a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

**Answer:**<br/>
A. This formula can be used to calculate the gradient of any function f (differentiable) and specifically ANN function numerically in the following way:<br/>
We define  ${g(\Delta x;x_0) = \frac{f(x_0 + \Delta x) - f(x_0)}{\Delta x}}$  and the gradient of f at ${x_0}$ will be ${lim_{\Delta x \rightarrow 0}g(\Delta x;x_0)}$.<br/>
In order to calculate this we would define a series ${\{d_0,d_1=d_0/2,d2=d1/2,...\} s.t :d_0}$ is small. <br/>
We would try and find an index i such that ${g(d_i;x_0)\approx g(d_{i+1};x_0)}$ and the gradient of f at ${x_0}$ will approximately be ${g(d_i;x_0)}$.<br/>
B. One of the main drawbacks of this approach is that it is hard to calculate the gradient with this approach for functions with high curvature at ${x_0}$ since the gradient function is changing at ${x_0}$ at thus this approach will require smaller ${\Delta x}$ to calculate the gradient.<br/>
Another main drawback is the numerical instability of this method as it is dependant on calculating the division of a very small value by a very small value, thus the value calculated by a program of ${g(\Delta x;x_0}$ is uncertain to represent to analytical value of this function when ${\Delta x}$ is very small.

3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [3]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
grad_W = torch.zeros_like(W)
grad_b = torch.zeros_like(b)
d0 = 10**-2
eps = 10**-6


tempW = W.clone()
tempb = b.clone()
for i in range(d):   
    for j in range (d):
        d1 = d0/2
        #calculating g(d0;0)
        tempW[i][j] += d0
        g0 = (foo(tempW, b) - foo(W, b)) / d0
        tempW[i][j] -= d0
        #calculating g(d1;0)
        tempW[i][j] += d1
        g1 = (foo(tempW, b) - foo(W, b)) / d1
        tempW[i][j] -= d1
        while (torch.abs(g1-g0) > eps).any().item():
            g0 = g1
            d1 = d1/2
            tempW[i][j] += d1
            g1 = (foo(tempW, b) - foo(W, b)) / d1
            tempW[i][j] -= d1
        
        grad_W[i][j] = g1
    d1 = d0/2
    #calculating g(d0;0)
    tempb[i] += d0
    g0 = (foo(W, tempb) - foo(W, b)) / d0
    tempb[i] -= d0
    tempb[i] += d1
    g1 = (foo(W, tempb) - foo(W, b)) / d1
    tempb[i] -= d1
    while (torch.abs(g1-g0) > eps).any().item():
            g0 = g1
            d1 = d1/2
            tempb[i] += d1
            g1 = (foo(W, tempb) - foo(W, b)) / d1
            tempb[i] -= d1
    grad_b[i] = g1


# TODO: Compare with autograd using torch.allclose()
autograd_W = torch.autograd.grad(loss, W)[0]
autograd_b = b_autograd = torch.autograd.grad(loss, b)[0]
assert torch.allclose(grad_W, autograd_W)
assert torch.allclose(grad_b, autograd_b)

loss=tensor(1.4470, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?

**Answer:**<br/>
A. Word embedding is a mapping of words from a certain language space to embedded latent space. In this latent space, the relation between words is preserved i.e the distance between words in the embdded space represents their similiarity in the space we are learning. <br/>
B. Theoretically a language model can be trained without an embedding, the consquences for the trained model are that we loose the processing of the embedding layer, and thus the difference between words representation will be independant of their similiarity, this will lead to degradation in performance as the representation capcity of the model decreases (the task of the recurrent layer is harder).

2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [3]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


**Answer:**<br/>
A. Y contains the embedding of a 4 dimensional tensor X, where each element of X: ${x \in \{\alpha_1,...,\alpha_{42}\}}$ embedded to dimension 42,000.
The output shape is 5,6,7,8,42000 since each word in a tensor of shape 5,6,7,8 is embedded to a vector of size 42,000.<br/>
B. There are several ways to implement 'nn.Embedding', we would take X, create a new tensor ${X'}$ of shape 5,6,7,8,1 (by unsqueeze over X)and we would multiply this ${X'}$ tensor by a vector ${w}$ of shape 1,42000. the vector ${w}$ is a learnable parameter.

3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

**TODO: CHECK SECTION B**

**Answer:**<br/>
A. True. This is a variation of the backpropagation algorithm were we do the forward pass ${k_1}$ times and do backpropagation after ${k_2}$ timesteps.<br/>
B. False. While the sequence S effects the hyperparameters of the TBPTT algorithm, but does not necessarily satisfies it.<br/>
C. False. The memory is longer than the sequence length simply beacuse we have a hidden state. The hidden state encode the context of the model, so it contain the necessary context from the previus sequences as well.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


**TODO: reviwe Section B**

**Answer:**<br/>
A. Attention mechanism uses the encoder's final hidden states as 'keys' and the decoder's hidden state as queries.<br/>The input to the decoder is concatanted with a weighted average of the encoder hidden state. This process have the notion of allowing the model to focus (i.e put attention) to specific words in the input. The encoder hidden states will represent the words the model focuses on.<br/>
B. self attention is better at modeling dependencies between different parts of the same sequence. while attention on the other hand models only the dependencies between 2 different sequences.

### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

**Answer:**<br/>
A. The effect of the absence of the KL-divergence from the loss function on the Images reconstructed by the model during training will be that the reconstuction will be 'better' i.e the reconsturction loss obtained after optimization will be lower. That is, becuase instead of optimizing a loss function comprised of a sum of two loss functions (including reconstruction loss), we would optimize only on the reconstruction loss, and thus will probably be able to obtain the absolute minimum reconstruction loss over the training images, whie training over the regular loss function will not necesserily find this minimum as the minimum of the sum of both loss terms.<br/>
B. The effect on the images generated by the model will be less similiar to the images we trained on. That is, since the latent space could be distributed 'wildly' i.e not similiar to a known distribution and specifically not necesserily similiar to Normal distribution. Thus, our sampling from the latent space would not necesserily represent a latent value mapping for a real image and therefore after decoding we would probably get an image that is far from being similiar to our original images the model trained on.

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

**Answer:**<br/>
A. **False**. We measure the similiarity between the latent-space distribution given input and ${N(0,I)}$ (which is the prior latent space distribution) thus we are not guaranteed that this KL-divergence between these two distributions will be zero and thus the latent space distribtuion generated by the model for a specific input is not ${N(0,I)}$.<br/>
B. **True**. After training, the encoder and decoder are both compirsed of fixed weights of convolutions and transpose convolutions accordingly, thus they are both determenistic and thus, encoding the same input will always provide the same latent-space vector, and decoding that same vector will always return the same output image.<br/>
C. **True**. The VAE loss is: $\mathcal{L} = -\mathbb{E}_{\bb{x}} \log p(\bb{X})$, this expectation term is intractable, yet we can use the following inequality: $$
\log p(\bb{X}) \ge \mathbb{E} _{\bb{z} \sim q _{\bb{\alpha}} }\left[ \log  p _{\bb{\beta}}(\bb{X} | \bb{z}) \right]
-  \mathcal{D} _{\mathrm{KL}}\left(q _{\bb{\alpha}}(\bb{Z} | \bb{X})\,\left\|\, p(\bb{Z} )\right.\right)
$$ and optimize this upper bound of the VAE loss.


2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
  1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
  1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?